In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import os
import numpy as np
import pandas as pd
import dateutil
import datetime
import re
import matplotlib.pyplot as plt
import math

import load_data

import pecos

from bokeh.io import output_notebook, show
from bokeh.plotting import figure
from bokeh.palettes import Category10
from bokeh.layouts import gridplot
from bokeh.models import Legend, HoverTool, ColumnDataSource
output_notebook()

# pd.set_option('display.height', 1000)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
# pd.set_option('display.width', 1000)
#test

In [ ]:
raw_data = load_data.load_data(directory='./data_b/')

trans = load_data.trans_dict(raw_data)

pm = pecos.monitoring.PerformanceMonitoring()
system_name = 'test'
pm.add_dataframe(raw_data, system_name)
pm.add_translation_dictionary(trans, system_name)

In [ ]:
raw_data.shape

In [ ]:
raw_data.head(1)

In [ ]:
grid = load_data.plot(pm)
show(grid)

In [ ]:
list(trans.keys())

In [ ]:
raw_data.plot(kind='scatter', y='Blythe2_SUB1_MET1_POA-Reference Cell W/m2', x='Blythe2_SUB1_MET1_Temperature-Ambient �C',
                         title='Actual Test Data Set', ylim=(0,1500), xlim=(0,50), alpha=0.2)

In [ ]:
# hover = HoverTool(tooltips=[('Date', '@index')])

# source = ColumnDataSource(pm.df)

# #x = pm.df[pm.trans['index--']]
# y = 'test:component Canal Solar - KZPOA - Measured Irradiance Other, avg, (W/m&#178;)'

# p = figure(plot_width=700, plot_height=300, x_axis_type='datetime')
# p.add_tools(hover)
# line = p.line(x='test:index', y=y, source=source)

# output_notebook()
# show(p)

hover = HoverTool(tooltips=[('Date', '@index1')])

source = ColumnDataSource(raw_data)

#x = pm.df[pm.trans['index--']]
x = 'index'
y = 'component Canal Solar - PVEL - Measured Irradiance Plane of Array, avg, (W/m&#178;)'

p = figure(plot_width=700, plot_height=300, x_axis_type='datetime')
p.add_tools(hover)
line = p.line(x=x, y=y, source=source)

output_notebook()
show(p)

In [ ]:
# Belongs in Section 9.1.5 DAS Equipment Malfunction

# raw_data_sensors_filtered = load_data.apply_filter(pm, perc_diff=.05, skip_strs=['op_state', 'real_pwr-valuesError-mtr-', 'wind'])
# dtindex = pd.DatetimeIndex(freq='5Min', start=raw_data_sensors_filtered.index.tolist()[0], end=raw_data_sensors_filtered.index.tolist()[-1])
# filtered = raw_data_sensors_filtered.reindex(index=dtindex)
# trans2 = load_data.trans_dict(raw_data_sensors_filtered)
# pm2 = pecos.monitoring.PerformanceMonitoring()
# pm2.add_dataframe(filtered, 'clean')
# pm2.add_translation_dictionary(trans2, 'clean')
# grid2 = load_data.plot(pm2)
# show(grid2)